In [ ]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWINB", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWINB", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################


# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 4
Maxiterations_turbine = 30

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True
# Расчет для работы с теплофикацией
Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
PKM_zaryad = True
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

# Расчет всей ПГУ вместе


def calculate_CCGT(Iterations_KU_TU, Iterations_cotel, Iterations_turbine):

    # Расчет ГТУ
    Gas_turbine_res = Gas_turbine.calc(GTU_input)

    # Запись данных об электричестве
    electric.at["GTU", "N"] = Gas_turbine_res["N"]
    electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
    electric.at["DK", "N"] = Gas_turbine_res["Ndk"]

    # Запись данных о газе на выходе из ГТУ
    gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
    gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
    gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
    gas_streams.at["GTU-KU", "P"] = 0.1
    gas_streams.at["GTU-KU", "H"] = gas0.p_t(
        gas_streams.at["GTU-KU", "P"], gas_streams.at["GTU-KU", "T"]
    )["h"]
    Gas_turbine_composition = pd.read_excel(
        "input.xlsx", sheet_name="Gas_composition0", index_col=0
    )

    #####################Максимов#####################
    from PKM import PKM_all

    PKM = PKM_all.calc(
        PKM_zaryad, gas_streams, syngas_streams, water_streams, water_streams0, heaters
    )
    steamVD_to_turbine = PKM["steamVD_to_turbine"]
    ##################################################

    # Состав газов при частичной нагрузке
    fractiongas = list(gas_streams.loc["GTU-PEVD", "N2":"Ar"])

    gas1 = prop.Materials_prop(
        gasmix,
        fractiongas,
        prop.REFPROP_h_s,
        prop.REFPROP_p_t,
        prop.REFPROP_p_h,
        prop.REFPROP_p_s,
        prop.REFPROP_p_q,
        prop.REFPROP_t_q,
        prop.REFPROP_p_rho,
        prop.REFPROP_s_q,
        RP=RP,
    )

    # Инициализаця KU+TU, она здесь потому что нужно менять состав газа на входе в КУ

    KU_and_TU = KU_TU.ku_tu(
        gas0,
        gas1,
        water,
        gas_streams0,
        gas_streams,
        water_streams0,
        water_streams,
        heaters,
        electric,
        "PEVD-DROSVD",
        "PPND-DROSND",
        "DROSVD-TURBVD",
        "DROSND-TURBND",
        Calcmethod,
        KPD_SP,
        KPD_KN,
        KPD_to,
        KPD_PN,
        steamVD_fraction_to_turbine,
        steamVD_to_turbine,
    )

    start_time = time.time()

    # Расчет КУ и ТУ
    KU_and_TU.calculate(
        Teplo,
        Calctolerance,
        Iterations_KU_TU,
        Iterations_cotel,
        Iterations_turbine,
    )

    print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")


Iter_pkm = 20
pkm_pgu_tol = 10**-3

start_time = time.time()
Gst = [water_streams0.at["DROSVD-ST", "G"]]
Qref = [1]
Ggpk = [water_streams0.at["SMESH-GPK", "G"]]

for i in range(Iter_pkm):
    if i < 7:
        Maxiterations_KU_TU_new = int(2)
        Maxiterations_cotel_new = int(3)
        # print(Maxiterations_KU_TU_new,Maxiterations_cotel_new)

    else:
        Maxiterations_KU_TU_new = Maxiterations_KU_TU
        Maxiterations_cotel_new = Maxiterations_KU_TU

    calculate_CCGT(
        Maxiterations_KU_TU_new, Maxiterations_cotel_new, Maxiterations_turbine
    )

    print(
        f"Время {i+1} итерации расчета КУ+ТУ с ПКМ:--- %s сек. --- {round((time.time() - start_time), 1)}"
    )
    
    Gst.append(round(water_streams.at["DROSVD-ST", "G"], 2))
    Qref.append(round(heaters.loc["Ref_all", "Qw"], 2))
    Ggpk.append(round(water_streams.at["SMESH-GPK", "G"], 2))
    
    print("Gst", Gst)
    print("Qref", Qref)
    print("Ggpk", Ggpk)
    
    Err1 = abs((Gst[i] - Gst[i - 1]) / (Gst[i]) * 100)
    Err2 = abs((Qref[i] - Qref[i - 1]) / (Qref[i]) * 100)
    Err3 = abs((Ggpk[i] - Ggpk[i - 1]) / (Ggpk[i]) * 100)

    if i == Iter_pkm - 1:
        print("Достигнуто максимальнео количество итераций КУ+ПТУ+ПКМ")

    if Err1 < pkm_pgu_tol and Err2 < pkm_pgu_tol:

        print(
            f"Расчет КУ+ПТУ+ПКМ закончен:--- %s сек. ---{round((time.time() - start_time), 1)}"
        )
        print(Err1, Err2, Err3)

        break

Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 1.1205812336756584e-14, Error_nd_G: 0.009530640931134825, Error_vd_G: 0.0
Время 1 итерации расчета КУ+ТУ:---  28.0 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 2.349803420930347e-14, Error_nd_G: 0.0006105893052970793, Error_vd_G: 0.0
Время 2 итерации расчета КУ+ТУ:---  52.9 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ 2
Pnd_it [0.56396, 0.46714]
Pvd_it [5.8763, 4.55544]
fin КУ и ТУ:--- 52.9 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ:--- %s сек. --- 53.4
Gst [nan, 47.55]
Qref [1, 176785.52]
Ggpk [78.16, 77.79]
Достигнуто максимальное количество итераций контура высокого давлени

In [ ]:
gas_streams

In [ ]:
gas_streams0

In [ ]:
water_streams

In [ ]:
heaters

In [ ]:
syngas_streams